In [1]:
import torch
from datasets import load_dataset
from transformers import (
    MBart50TokenizerFast,
    MBartForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)

In [2]:
# --- Configuração ---
MODEL_CHECKPOINT = "facebook/mbart-large-50-many-to-many-mmt"
SOURCE_LANG = "en_XX" # Inglês
TARGET_LANG = "pt_XX" # Português
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128

In [3]:
# Um córpus em paralelo com diversas línguas
dataset = load_dataset("opus_books", "en-pt")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 1404
    })
})

In [4]:
# Use o train_test_split para dividir em treino e teste
dataset = dataset["train"].train_test_split(test_size=0.2)

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 1123
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 281
    })
})

In [6]:
# Espiadinha nos dados
print(dataset["train"][0].keys())

print(dataset["train"][0]["translation"].keys())

print("\ten:", dataset["train"][0]["translation"]["en"])
print("\tpt:", dataset["train"][0]["translation"]["pt"])

dict_keys(['id', 'translation'])
dict_keys(['en', 'pt'])
	en: Alice crouched down among the trees as well as she could, for her neck kept getting entangled among the branches, and every now and then she had to stop and untwist it.
	pt: Alice agachou-se entre as árvores o melhor que pode, pois o pescoço dela continuou se enredando entre os galhos, e aqui e ali ela tinha que parar e desenredá-lo.


In [7]:
tokenizer = MBart50TokenizerFast.from_pretrained(MODEL_CHECKPOINT)

def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["pt"] for ex in examples["translation"]]

    # 1. Definir a língua fonte (Essencial para o mBART)
    tokenizer.src_lang = SOURCE_LANG
    
    # 2. Definir a língua alvo (Essencial para o mBART)
    tokenizer.tgt_lang = TARGET_LANG

    # 3. Tokeniza o texto de entrada (texto em inglês)
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)

    # 4. Tokenize o texto alvo (portugês)
    labels = tokenizer(
        text_target=targets, 
        max_length=MAX_TARGET_LENGTH, 
        truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Mapeia a função de tokenização no dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1123 [00:00<?, ? examples/s]

Map:   0%|          | 0/281 [00:00<?, ? examples/s]

# Fine-tuning

In [8]:
# 1. Carrega o modelo base
model = MBartForConditionalGeneration.from_pretrained(
    MODEL_CHECKPOINT,
    device_map="auto" 
)

In [9]:
model

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        

In [10]:
# help(model.base_model.model.forward)

# Opcional utilizar peft para realizar LoRa
Utilizando a biblioteca [PEFT](https://huggingface.co/docs/peft/index)

In [11]:
from peft import LoraConfig, get_peft_model, TaskType
# 2. Define a configuração do LoRA
lora_config = LoraConfig(
    r=16, 
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"], # Aplica nas camadas de atenção
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

# 3.Envelopa o modelo com PEFT
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
# Os parâmetros treináveis devem ser aproximadamente 0.5% ou menos

trainable params: 2,359,296 || all params: 613,238,784 || trainable%: 0.3847


# Configuração de avaliação do modelo
- Utilizando somente [BLEU](https://huggingface.co/spaces/evaluate-metric/bleu)
- Diferença entre sacrebleu e bleu [A Call for Clarity in Reporting BLEU Scores](https://arxiv.org/abs/1804.08771)

In [16]:
import evaluate
import numpy as np

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    # Para caso o modelo returne mais do que os logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Substitui -100 nos rótulos já que não dá para decodificá-los
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU recebe uma lista de listas para referências
    decoded_labels = [[l] for l in decoded_labels]

    # Aqui vocês podem adicionar outras métricas
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) 

training_args = Seq2SeqTrainingArguments(
    output_dir="./mbart50-finetuned",
    learning_rate=1e-4, # 5e-5 Consultar a documentação / artigo do modelo
    per_device_train_batch_size=4, # Ideal é aumentar até chegar no limite da memória
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    predict_with_generate=True, 
    bf16=True, # Treina com precisão mais baixa, porém mais rápida
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

train_output = trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
1,No log,1.606027,32.593299


In [20]:
train_output

TrainOutput(global_step=281, training_loss=1.6249434175864657, metrics={'train_runtime': 43.8167, 'train_samples_per_second': 25.63, 'train_steps_per_second': 6.413, 'total_flos': 125986729574400.0, 'train_loss': 1.6249434175864657, 'epoch': 1.0})

In [25]:
# Número de teraflops utilizados
# Número total operaçõeS de número de ponto-flutuante (total_flos)
print(train_output.metrics["total_flos"] / (train_output.metrics["train_runtime"] *  1e12))

2.8753130558531335


# Gera Tradução

In [28]:
import ipywidgets as widgets
from IPython.display import display

input_text = widgets.Textarea(
    value="The students are learning about natural language processing today.",
    placeholder="Type English text here...",
    layout={'width': '100%'}
)
button = widgets.Button(description="Translate", button_style='primary')
output = widgets.Output()

def on_click(b):
    output.clear_output()
    with output:
        # Garante que a língua fonte está definida
        tokenizer.src_lang = SOURCE_LANG 
        
        # Tokeniza e move para o mesmo dispositivo onde o modelo está model (GPU/CPU)
        inputs = tokenizer(input_text.value, return_tensors="pt").to(model.device)
        
        # Gera o texto forçando a língua alvo
        generated_tokens = model.generate(
            **inputs, 
            forced_bos_token_id=tokenizer.lang_code_to_id[TARGET_LANG]
        )
        
        # Decodifica e imprime
        result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        print(f"Prediction: {result}")

button.on_click(on_click)
display(widgets.VBox([input_text, button, output]))